In [3]:
# For langchain
!pip install langchain-openai langchain-experimental -q

# For ArangoDB
!pip install python-arango -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 4.8 MB/s eta 0:00:00


In [32]:
!pip install matplotlib

     ---------------------------------------- 8.1/8.1 MB 17.2 MB/s eta 0:00:00
     ------------------------------------- 219.8/219.8 kB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 23.3 MB/s eta 0:00:00
     ---------------------------------------- 72.0/72.0 kB ? eta 0:00:00
     ---------------------------------------- 107.7/107.7 kB ? eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import os

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

In [25]:
from arango import ArangoClient
import networkx as nx
import nx_arangodb as nxadb

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root",
        password="RHr0KzkRUVlp61IisH8G",
        name="DAC_devops_log",
        verify=True)

In [26]:
task_dep_graph = nxadb.DiGraph(name="tasks_sprint1")

[00:27:50 -0500] [INFO]: Graph 'tasks_sprint1' exists.
[00:27:50 -0500] [INFO]: Default node type set to 'task'


In [27]:
print(task_dep_graph.nodes)

['task/TSK-5C2187CF', 'task/TSK-EBA1C369', 'task/TSK-C4D5CB75', 'task/TSK-A8A24727', 'task/TSK-CB292DC1', 'task/TSK-CB0C0404', 'task/TSK-29CE08AD', 'task/TSK-89BBB564', 'task/TSK-8CD3CA3E', 'task/TSK-A0500206', 'task/TSK-39885F7F', 'task/TSK-FAC99DF1', 'task/TSK-0E209EFC', 'task/TSK-0115FED3', 'task/TSK-255E6310', 'task/TSK-1AA8BEAF', 'task/TSK-5D974201', 'task/TSK-84305F2F', 'task/TSK-01D25D85', 'task/TSK-BE9555AC', 'task/TSK-B03EF789', 'task/TSK-BE4188D4', 'task/TSK-0985DBA9', 'task/TSK-59CE546C', 'task/TSK-C891B895', 'task/TSK-54F9521F', 'task/TSK-BBA87B09', 'task/TSK-514F6016', 'task/TSK-93F18D57', 'task/TSK-3D21853C', 'task/TSK-2B1FAB57', 'task/TSK-D5031D4A', 'task/TSK-DC3ED3E2', 'task/TSK-B600DD62', 'task/TSK-427652F3', 'task/TSK-0885F3E9', 'task/TSK-1FE396B7', 'task/TSK-19A45EE8', 'task/TSK-E6CFDD02', 'task/TSK-7E9F09A9', 'task/TSK-736E3B28', 'task/TSK-4A3587C9', 'task/TSK-2970AC03', 'task/TSK-E2842052', 'task/TSK-E2E87C0B', 'task/TSK-983C30DC', 'task/TSK-10847035', 'task/TSK-AE

In [17]:
task_col = db.collection("task")

print(task_col.all())
for task in task_col.all():
    print(task)

<Cursor>
{'_key': 'TSK-FAC99DF1', '_id': 'task/TSK-FAC99DF1', '_rev': '_jTkZs4q--_', 'TaskID': 'TSK-FAC99DF1', 'Description': 'Deployment to production environment', 'StoryPoints': '5', 'Status': 'Planned', 'size': 10, 'duration': 5}
{'_key': 'TSK-E2842052', '_id': 'task/TSK-E2842052', '_rev': '_jTkZtFi--A', 'TaskID': 'TSK-E2842052', 'Description': 'Performing a performance test on the server', 'StoryPoints': '7', 'Status': 'Planned', 'size': 10, 'duration': 7}
{'_key': 'TSK-FAAA4D51', '_id': 'task/TSK-FAAA4D51', '_rev': '_jTkZtUe---', 'TaskID': 'TSK-FAAA4D51', 'Description': 'Conduct initial data analysis to identify data types, data volume, and data anomalies', 'StoryPoints': '3', 'Status': 'Planned', 'size': 10, 'duration': 3}
{'_key': 'TSK-A3E27D20', '_id': 'task/TSK-A3E27D20', '_rev': '_jTkZtky--_', 'TaskID': 'TSK-A3E27D20', 'Description': 'Preparation for deployment', 'StoryPoints': '2', 'Status': 'Planned', 'size': 10, 'duration': 2}
{'_key': 'TSK-C440EB07', '_id': 'task/TSK-C44

In [11]:
from collections import deque


def layered_topo_sort(G):
    """
    Performs a topological sort on a directed acyclic graph (DAG) using the peeling technique.
    
    :param G: Directed graph (DiGraph) from NetworkX
    :return: A list of layers, where each layer is a list of nodes.
    """
    # Ensure the graph is a DAG
    if not nx.is_directed_acyclic_graph(G):
        raise ValueError("The graph is not a Directed Acyclic Graph (DAG)")
    
    # In-degree calculation: count the number of incoming edges for each node
    in_degree = {node: G.in_degree(node) for node in G.nodes}
    
    layers = []
    
    # Use a queue to keep track of nodes with zero in-degree (ready to be processed)
    queue = deque([node for node, degree in in_degree.items() if degree == 0])
    
    while queue:
        layer = []
        
        # Process all nodes with zero in-degree at the current level
        for _ in range(len(queue)):
            node = queue.popleft()
            layer.append(node)
            
            # For all outgoing edges, reduce in-degree of target nodes
            for neighbor in G.neighbors(node):
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    queue.append(neighbor)
        
        # Add the processed layer to the result
        layers.append(layer)
    
    return layers

In [15]:
print(task_dep_graph.nodes)

[]


In [12]:
layered_sort = layered_topo_sort(task_dep_graph)

In [10]:
print(layered_sort[0])

IndexError: list index out of range

In [22]:
task_dep_graph.in_edges(layered_sort[-1][0])

InEdgeDataView([('task/TSK-29CE08AD', 'task/TSK-5A434477'), ('task/TSK-0985DBA9', 'task/TSK-5A434477'), ('task/TSK-A0500206', 'task/TSK-5A434477'), ('task/TSK-B03EF789', 'task/TSK-5A434477'), ('task/TSK-FBD87AD9', 'task/TSK-5A434477')])

In [28]:
import networkx as nx
import random

def color_dag(dag):
    # Perform topological sorting
    topo_order = list(nx.topological_sort(dag))
    
    # Initialize dictionary for layers
    layers = {}
    
    # Perform layering of the nodes (BFS-like approach)
    # We'll use a dictionary to track the minimum distance of each node from the source nodes
    for node in topo_order:
        # If the node has no incoming edges, it's a source, so assign to layer 0
        if dag.in_degree(node) == 0:
            layers[node] = 0
        else:
            # Assign to the layer of the maximum incoming node + 1
            incoming_layers = [layers[neighbor] for neighbor in dag.predecessors(node)]
            layers[node] = max(incoming_layers) + 1

    print(layers)
    # Find the max layer value (bottom layer)
    max_layer = max(layers.values())
    
    # Create color mapping
    node_colors = {}
    for node, layer in layers.items():
        if layer == 0:
            node_colors[node] = "Red"  # Topmost layer
        elif layer == max_layer:
            node_colors[node] = "Blue"  # Bottom layer
        else:
            node_colors[node] = "Green"  # Intermediate layers
    
    # Shuffle the nodes in each layer for randomness
    layer_nodes = {layer: [] for layer in range(max_layer + 1)}
    for node, layer in layers.items():
        layer_nodes[layer].append(node)

    # Randomize node order in each layer
    for layer in layer_nodes:
        random.shuffle(layer_nodes[layer])

    # Rebuild node_colors with randomized positions per layer
    randomized_node_colors = {}
    for layer in range(max_layer + 1):
        for i, node in enumerate(layer_nodes[layer]):
            randomized_node_colors[node] = node_colors[node]
    
    return randomized_node_colors, layers


# Example usage:

# Create a DAG
dag = nx.DiGraph()
dag.add_edges_from([
    ('A', 'B'),
    ('A', 'C'),
    ('B', 'D'),
    ('C', 'D'),
    ('D', 'E'),
])

# Call the function to color the DAG
node_colors, layers = color_dag(task_dep_graph)

# Print the results
print("Node colors based on layers:")
for node, color in node_colors.items():
    print(f"Node {node} is {color}")

print("\nLayer assignments:")
for node, layer in layers.items():
    print(f"Node {node} is in layer {layer}")

[00:28:04 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.3997955322265625s


{'task/TSK-93F18D57': 0, 'task/TSK-AAF06E6A': 0, 'task/TSK-2970AC03': 0, 'task/TSK-BBA87B09': 0, 'task/TSK-A3E27D20': 0, 'task/TSK-46138F7D': 0, 'task/TSK-3D21853C': 0, 'task/TSK-DA566BCB': 0, 'task/TSK-1AA8BEAF': 0, 'task/TSK-91119F5B': 0, 'task/TSK-01D25D85': 0, 'task/TSK-DC3ED3E2': 0, 'task/TSK-0AEE35B2': 0, 'task/TSK-2179FF75': 0, 'task/TSK-BE4188D4': 0, 'task/TSK-89806FDF': 0, 'task/TSK-30DBEA24': 0, 'task/TSK-39885F7F': 0, 'task/TSK-0885F3E9': 0, 'task/TSK-D5031D4A': 0, 'task/TSK-E2E87C0B': 0, 'task/TSK-BE9555AC': 0, 'task/TSK-514F6016': 0, 'task/TSK-FEA63216': 0, 'task/TSK-0E209EFC': 0, 'task/TSK-5D974201': 0, 'task/TSK-991E6E11': 0, 'task/TSK-C440EB07': 0, 'task/TSK-FBED5511': 0, 'task/TSK-84305F2F': 0, 'task/TSK-1A209469': 1, 'task/TSK-977CB080': 1, 'task/TSK-4A3587C9': 1, 'task/TSK-FD3B1EFF': 1, 'task/TSK-2B1FAB57': 1, 'task/TSK-8CD3CA3E': 1, 'task/TSK-CAFF3B09': 1, 'task/TSK-4D677758': 1, 'task/TSK-255E6310': 1, 'task/TSK-B600DD62': 1, 'task/TSK-89754CBF': 1, 'task/TSK-F51C1

In [ ]:
def assign_nodes(G):
    layers = nx.topological_generations(G)

    assignment = {node: "Planned" for node in G.nodes}
    completed_probability = 1
    layers = [sorted(generation) for generation in layers]
    length = len(layers)
    for level, nodes in enumerate(layers):
        for node in nodes:
            if random.random() < completed_probability:
                if random.random() < 0.3:
                    assignment[node] = "Blocked"
                else:
                    assignment[node] = "In Progress"
            else:
                assignment[node] = "Completed"
        completed_probability -= 1 / (length)

    return assignment

print(assign_nodes(task_dep_graph))


0
0.8888888888888888
1
0.7777777777777777
2
0.6666666666666665
3
0.5555555555555554
4
0.44444444444444425
5
0.33333333333333315
6
0.22222222222222204
7
0.11111111111111094
8
-1.6653345369377348e-16
